In [1]:
import pandas as pd
import json
import requests

In [2]:
# Opening JSON file
f = open("/Users/mio/Documents/GitHub/aging-society/data/facility_data_ver2.json")
data = json.loads(f.read())

In [3]:
df = pd.json_normalize(data['cities'])

In [4]:
df.columns

Index(['code', 'population.age40_64.2020', 'population.age40_64.2025',
       'population.age40_64.2030', 'population.age40_64.2035',
       'population.age40_64.2040', 'population.age40_64.2045',
       'population.over65.2020', 'population.over65.2025',
       'population.over65.2030', 'population.over65.2035',
       'population.over65.2040', 'population.over65.2045',
       'population.over75.2020', 'population.over75.2025',
       'population.over75.2030', 'population.over75.2035',
       'population.over75.2040', 'population.over75.2045',
       'population.demand.2020', 'population.demand.2025',
       'population.demand.2030', 'population.demand.2035',
       'population.demand.2040', 'population.demand.2045',
       'facilities.number.total', 'facilities.number.home-visit',
       'facilities.number.daycare', 'facilities.number.residential-care',
       'facilities.number.specified', 'facilities.number.home-care-support',
       'facilities.number.assistive-products', 'facilit

In [5]:
demand = df[['code','population.demand.2025','population.demand.2025','population.demand.2030','population.demand.2035','population.demand.2040','population.demand.2045']].copy()

In [6]:
demand

,code,population.demand.2025,population.demand.2025,population.demand.2030,population.demand.2035,population.demand.2040,population.demand.2045
0,01101,36312.640,36312.640,40954.477,44150.942,47809.810,51505.615
1,01102,47146.376,47146.376,52412.093,53946.723,55042.915,56628.014
2,01103,41289.623,41289.623,45653.784,46881.670,48005.456,49879.018
3,01104,32540.141,32540.141,35868.262,37300.926,38975.467,40886.605
4,01105,35426.074,35426.074,38843.193,40412.988,42107.606,43981.430
...,...,...,...,...,...,...,...
1891,31384,609.943,609.943,655.616,682.933,679.825,680.379
1892,33101,48323.022,48323.022,50270.150,50479.007,50970.956,52302.228
1893,33204,12758.348,12758.348,12613.425,11547.688,10421.853,9500.106
1894,38422,3529.612,3529.612,3534.724,3427.533,3166.927,2850.532


In [7]:
response = requests.get("https://madefor.github.io/jisx0402/api/v1/all.json")
data = response.json()
cities = pd.DataFrame(data).T.rename_axis('code_6digit').reset_index()
cities['code_5digit'] = cities.code_6digit.str.extract(r'(.*)\d').astype(str)
cities

,code_6digit,prefecture,city,prefecture_kana,city_kana,code_5digit
0,011002,北海道,札幌市,ホッカイドウ,サッポロシ,01100
1,011011,北海道,札幌市中央区,ホッカイドウ,サッポロシチュウオウク,01101
2,011029,北海道,札幌市北区,ホッカイドウ,サッポロシキタク,01102
3,011037,北海道,札幌市東区,ホッカイドウ,サッポロシヒガシク,01103
4,011045,北海道,札幌市白石区,ホッカイドウ,サッポロシシロイシク,01104
...,...,...,...,...,...,...
1917,473618,沖縄県,久米島町,オキナワケン,クメジマチョウ,47361
1918,473626,沖縄県,八重瀬町,オキナワケン,ヤエセチョウ,47362
1919,473758,沖縄県,多良間村,オキナワケン,タラマソン,47375
1920,473812,沖縄県,竹富町,オキナワケン,タケトミチョウ,47381


In [8]:
demand['prefecture'] = demand.code.map(cities.set_index('code_5digit').prefecture)

families

In [9]:
url = "https://www.ipss.go.jp/pp-pjsetai/j/hpjp2019/gaiyo/gaiyo_tab.xls"

In [50]:
#total
total = pd.read_excel(url, sheet_name="表Ⅱ-1", skiprows=3, header=0).iloc[2:50,1:8]
total = total.replace('全　 国','total').rename(columns={'Unnamed: 1':'prefecture'}).set_index('prefecture')

In [52]:
#over75-1: living on their own or with partners
over75_single = pd.read_excel(url, sheet_name="表Ⅱ-17-1", skiprows=3, header=0).iloc[2:50,:7]
over75_single = over75_single.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over75_couple = pd.read_excel(url, sheet_name="表Ⅱ-17-2", skiprows=3, header=0).iloc[2:50,:7]
over75_couple = over75_couple.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over75 = over75_single + over75_couple
over75_pct = over75 / total *100

over75_pct.to_csv('./data/families_over75.csv')

In [65]:
#over75-2: living with children and other members
over75_child1 = pd.read_excel(url, sheet_name="表Ⅱ-17-3", skiprows=3, header=0).iloc[2:50,:7]
over75_child1 = over75_child1.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over75_child2 = pd.read_excel(url, sheet_name="表Ⅱ-17-4", skiprows=3, header=0).iloc[2:50,:7]
over75_child2 = over75_child2.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over75_others = pd.read_excel(url, sheet_name="表Ⅱ-17-5", skiprows=3, header=0).iloc[2:50,:7]
over75_others = over75_others.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over75_others = over75_child1 + over75_child2 + over75_others
over75_others_pct = over75_others / total *100

In [56]:
#over65
over65_single = pd.read_excel(url, sheet_name="表Ⅱ-10-1", skiprows=3, header=0).iloc[2:50,:7]
over65_single = over65_single.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over65_couple = pd.read_excel(url, sheet_name="表Ⅱ-10-2", skiprows=3, header=0).iloc[2:50,:7]
over65_couple = over65_couple.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over65 = over65_single + over65_couple
over65_pct = over65 / total *100

over65_pct.to_csv('./data/families_over65.csv')

In [77]:
#over65-2: living with children and other members
over65_child1 = pd.read_excel(url, sheet_name="表Ⅱ-10-3", skiprows=3, header=0).iloc[2:50,:7]
over65_child1 = over65_child1.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over65_child2 = pd.read_excel(url, sheet_name="表Ⅱ-10-4", skiprows=3, header=0).iloc[2:50,:7]
over65_child2 = over65_child2.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over65_others = pd.read_excel(url, sheet_name="表Ⅱ-10-5", skiprows=3, header=0).iloc[2:50,:7]
over65_others = over65_others.replace('全　 国','total').rename(columns={'Unnamed: 0':'prefecture'}).set_index('prefecture')

over65_others = over65_child1 + over65_child2 + over65_others
over65_others_pct = over65_others / total *100

In [62]:
over65_74 = over65 - over75
over65_74_pct = over65_74 / total *100

In [80]:
over65_74_others = over65_others - over75_others
over65_74_others_pct = over65_74_others / total *100

In [96]:
younger = 100 - (over75_pct.loc['total'] + over75_others_pct.loc['total'] + over65_74_pct.loc['total'] + over65_74_others_pct.loc['total'])
younger

2015年    64.038185
2020年    61.843904
2025年    61.136635
2030年    60.254930
2035年    58.724375
2040年    55.823760
Name: total, dtype: float64

In [108]:
demography = pd.concat([
    over75_pct.loc['total'].rename('over75_own'),
    over75_others_pct.loc['total'].rename('over75_with_family'),
    over65_74_pct.loc['total'].rename('over65_74_own'),
    over65_74_others_pct.loc['total'].rename('over65_74_with_family'),
    younger.rename('others')
],axis=1)

In [103]:
import re

In [106]:
demography.index = [int(re.sub('\D','',idx)) for idx in demography.index]

In [107]:
demography

,over75_own,over75_with_family,over65_74_own,over65_74_with_family,others
2015,11.445558,5.210693,12.047055,7.258508,64.038185
2020,13.376700,5.889240,12.064778,6.825378,61.843904
2025,15.856524,6.775355,10.522469,5.709016,61.136635
2030,16.867686,6.996314,10.526320,5.354750,60.254930
2035,16.892517,6.814914,11.940079,5.628116,58.724375
2040,17.252106,6.727179,13.941223,6.255732,55.823760


In [112]:
demography['over75_own'] + demography['over65_74_own']

2015年    23.492614
2020年    25.441478
2025年    26.378993
2030年    27.394006
2035年    28.832595
2040年    31.193329
dtype: float64

In [113]:
demography['over75_own'] / (demography['over75_own'] + demography['over75_with_family'])

2015年    0.687163
2020年    0.694319
2025年    0.700628
2030年    0.706826
2035年    0.712541
2040年    0.719459
dtype: float64

In [114]:
demography['over65_74_own'] / (demography['over65_74_own'] + demography['over65_74_with_family'])

2015年    0.624020
2020年    0.638681
2025年    0.648275
2030年    0.662822
2035年    0.679642
2040年    0.690264
dtype: float64

In [118]:
over75_single

,2015年,2020年,2025年,2030年,2035年,2040年
prefecture,,,,,,
total,3368.633,3958.385,4699.627,5045.209,5074.844,5122.158
北海道,179.292,205.551,242.874,265.225,266.154,262.303
青森県,34.406,37.436,43.247,47.429,48.637,48.492
岩手県,30.974,33.707,38.255,42.482,44.677,45.420
宮城県,50.026,57.248,69.265,80.073,85.314,87.458
秋田県,29.546,31.208,35.209,38.798,40.247,39.736
山形県,21.684,23.273,26.816,30.408,32.380,32.941
福島県,44.579,48.778,56.990,64.960,68.932,69.980
茨城県,54.624,66.181,82.321,93.323,96.451,97.489


In [110]:
demography.to_csv("./data/demograhy.csv")